In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import nltk
import tensorflow as tf
import os
import string
import re
import warnings
warnings.simplefilter("ignore")
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
data = pd.read_excel('sentence_past_present_future.xlsx')
y = data['Categorie']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y[0],y[1000],y[2700])
num_classes = len(np.unique(y))
one_hot_labels = to_categorical(y, num_classes=num_classes)
y = np.array(one_hot_labels)
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 10000
input = []
for x in data['Sentences']:
    val = word_tokenize(x)
    one_rpr = one_hot(str(val),n=voc_size,lower=False,split=None)
    input.append(one_rpr)
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
sent_length = 15
embedded_docs = pad_sequences(input,padding='post',maxlen=sent_length)
dim = 10
model =Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.compile('adam','mse')
model.summary()
x = model.predict(embedded_docs)
print(y.shape)
# rows = x.shape[0] 
# cols = x.shape[1] * x.shape[2]
# x = x.reshape(rows,cols)
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.25,random_state=42)
print(x_train.shape)
print(y_train.shape)

1 2 0
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 10)            100000    
                                                                 
Total params: 100000 (390.62 KB)
Trainable params: 100000 (390.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
87/87 [==============================] - 0s 987us/step
(2769, 3)
(2076, 15, 10)
(2076, 3)


In [98]:
lstm = Sequential([
     #Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length), # good for embedding
    LSTM(units=128,input_shape=(15,10),return_sequences= True),
    LSTM(units=64,activation='relu',return_sequences=True),
    LSTM(units=32,activation='relu'),
    Dense(units=3, activation='softmax')
])

In [99]:
lstm.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [100]:
lstm.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_28 (LSTM)              (None, 15, 128)           71168     
                                                                 
 lstm_29 (LSTM)              (None, 15, 64)            49408     
                                                                 
 lstm_30 (LSTM)              (None, 32)                12416     
                                                                 
 dense_15 (Dense)            (None, 3)                 99        
                                                                 
Total params: 133091 (519.89 KB)
Trainable params: 133091 (519.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [101]:
epochs = 100
lstm.fit(x_train,y_train, epochs=epochs)

Epoch 1/100
65/65 [==============================] - 6s 20ms/step - loss: 1.0856 - accuracy: 0.3979
Epoch 2/100
65/65 [==============================] - 1s 20ms/step - loss: 1.0580 - accuracy: 0.4398
Epoch 3/100
65/65 [==============================] - 1s 20ms/step - loss: 1.0439 - accuracy: 0.4504
Epoch 4/100
65/65 [==============================] - 1s 20ms/step - loss: 1.0204 - accuracy: 0.4894
Epoch 5/100
65/65 [==============================] - 1s 20ms/step - loss: 0.9881 - accuracy: 0.5352
Epoch 6/100
65/65 [==============================] - 1s 21ms/step - loss: 0.8961 - accuracy: 0.6108
Epoch 7/100
65/65 [==============================] - 1s 22ms/step - loss: 0.8062 - accuracy: 0.6546
Epoch 8/100
65/65 [==============================] - 1s 20ms/step - loss: 0.7951 - accuracy: 0.6590
Epoch 9/100
65/65 [==============================] - 1s 20ms/step - loss: 0.7639 - accuracy: 0.6643
Epoch 10/100
65/65 [==============================] - 1s 20ms/step - loss: 0.7412 - accuracy: 0.6802

In [102]:
lstm.evaluate(x_test,y_test)

22/22 [==============================] - 1s 8ms/step - loss: 0.6590 - accuracy: 0.7937


[0.6589720249176025, 0.7936508059501648]

In [103]:
def try_other_sentences(sentence):
    voc_size_val = 10000
    input_val = [sentence] 
    out_put = []
    for x in input_val:
        val = word_tokenize(x)
        one_rpr = one_hot(str(val),n=voc_size_val,lower=False,split=None)
        out_put.append(one_rpr)
    sent_length = 15
    embedded_sentence = pad_sequences(out_put,padding='post',maxlen=sent_length) #input- sentence
    value_to_pass = model.predict(embedded_sentence) 
    # rows = value_to_pass.shape[0]
    # cols = value_to_pass.shape[1] * value_to_pass.shape[2]
    # value_to_pass = value_to_pass.reshape(rows,cols) #coverting test sentence to same shape of 'x' . as we trained on.
    return value_to_pass

In [104]:
value = try_other_sentences('I ate an apple')

1/1 [==============================] - 0s 50ms/step


In [105]:
predictions = lstm.predict(value)

1/1 [==============================] - 1s 707ms/step


In [106]:
predict_class = np.argmax(predictions,axis=1)

In [107]:
print(predictions)

[[0.0030952  0.88366604 0.11323868]]


In [108]:
print(predict_class) #[1] index for past , [2] index for present , [0] index for future ....if o - past , 1 -present , 2-future

[1]


In [109]:
label_to_index = {"past": 1, "present": 2,"future":0}
index_to_label = {v: k for k, v in label_to_index.items()}

In [110]:
predicted_labels = [index_to_label[index] for index in predict_class]
print("\nPredicted Original Labels:", predicted_labels)


Predicted Original Labels: ['past']
